In [175]:
import random
import queue
import concurrent.futures
import math
from functools import reduce
from threading import Barrier

def share(secret, client_num):
    """
    Producing share based on the number of clients.
    """
    shares = [0] * client_num
    for i in range(client_num - 1):
        shares[i] = random.randint(1, 6)
    shares[client_num - 1] = secret - sum(shares[:client_num - 1])
    return shares

def A2B(A):
    neg = False
    if A < 0:
        neg = True
        A = -A
    B = []
    while A > 0:
        B.append(A % 2)
        A = math.floor(A / 2)
    B.append(0)
    if neg:
        c = 0
        for i in range(len(B)):
            if c:
                B[i] = 1 - B[i]
            else:
                c = B[i] == 1
    B.reverse()
    return B

def B2A(B):
    if B[0] == 0:
        return reduce(lambda a, b: 2*a + b, B)
    else:
        return -B2A([1 - b for b in B]) - 1

def CP(i, x, y, U, V, W, input_queue, output_queue, barrier):
    """
    The main function each party executes separately.
    """
    
    def mul_number(i, s, d):
        """
        Multiplication function based on Beaver triple.
        """
        D = s - U
        E = d - V

        # Sending and receiving intermediate results
        for q in output_queue:
            q.put((D, E))

        DE_values = [(D, E)] + [input_queue.get() for _ in range(len(output_queue))]

        D_sum = sum(D for D, E in DE_values)
        E_sum = sum(E for D, E in DE_values)

        Z = W + (D_sum * V) + (U * E_sum)

        if i == 0:
            Z += (D_sum * E_sum)
        return Z
    
    def o(d2, d1):
        """
        Carry propagation operator o.
        Computes a new (p, g) pair from two input pairs:
        (p, g) = (p2, g2) o (p1, g1) = (p1 & p2, g2 | (p2 & g1))
        """
        (p2, g2) = d2
        (p1, g1) = d1
        return (p1 & p2, g2 | (p2 & g1))
    
    def PreOpL(A):
        i = 1
        j = 2
        while j < len(A):
            for k in range(math.floor(len(A) / j)):
                p = i + k * j
                for q in range(i):
                    A[p+q] = o(A[p], A[p+q])
            i = j
            j *= 2
        return A

    def PreOpC(D):
        """
        Computes the carry bits from the input (p, g) pairs.
        The input consists of a (p, g) pair for each pair of bits to add.
        Each resulting pair is computed as (Pi, Gi) = (pi, gi) o (P[i-1], G[i-1])
        with (P0, G0) = (p0, g0).
        From each resulting pair, the carry bit can be obtained with ci = Gi.
        """
        p = D[0]
        P = [p]
        for i in range(1,len(D)):
            p = o(D[i], p)
            P.append(p)
        return P
    
    def addbitwise(A, B):
        """
        Performs bitwise addition on the given bit strings.
        The input parameters A and B are arrays of integers representing bits.
        The leftmost bits (A[0] and B[0]) represent the most significant bits.
        The output is a single array of bits representing the value A+B.
        """
        
        # Set the lengths of the bit strings to the next power of 2
        k = max(len(A),len(B)) + 1

        # Padding by extension of most significant bit
        A = [A[0]] * (k - len(A)) + A
        B = [B[0]] * (k - len(B)) + B

        A.reverse()
        B.reverse()

        # Calculate carry bits
        D = [(a + b - 2*a*b, a * b) for (a, b) in zip(A, B)]
        C = PreOpC(D)

        # Perform addition with carry bits
        S = [A[0] + B[0] - 2*C[0][1]]
        for i in range(1, k):
            S.append(A[i] + B[i] + C[i-1][1] - 2*C[i][1])
        S.reverse()

        # Debug output
        A.reverse()
        B.reverse()
        C.reverse()
        C = [c for (_, c) in C]
        print("A=", A)
        print("B=", B)
        print("S=", S, "cbit=", C[0])

        return S
    
    def CarryOutAux(D):
        k = len(D)
        if k > 1:
            U = []
            for i in range(1, 1 + math.floor(k/2)):
                U.append(o(D[2*i - 1], D[2*i - 2]))
            return CarryOutAux(U)
        else:
            return D[0]
    
    def CarryOutLin(D):
        return PreOpC(D)[len(D)-1]
    
    def CarryOutCin(A, B, c):
        pos = A[0] + B[0] == 0
        k = len(A)
        if k > len(B):
            B = [B[0] * (k - len(B))] + B
        elif k < len(B):
            A = [A[0] * (len(B) - k)] + A
            k = len(B)
        if pos:
            k -= 1
            A = A[1:]
            B = B[1:]

        A.reverse()
        B.reverse()

        # Calculate carry bits
        D = [(a + b - 2*a*b, a * b) for (a, b) in zip(A, B)]
        (p, g) = D[0]
        D[0] = (p, g + c*p)
        (_, g) = CarryOutLin(D)
        return g
    
    def BitLT(A, B):
        B = [1 - b for b in B]
        return 1 - CarryOutCin(A, B, 1)

    res  = B2A(addbitwise(A2B(x), A2B(y)))
    cbit = CarryOutCin   (A2B(x), A2B(y), 0)
    lt   = BitLT         (A2B(x), A2B(y))
    print(x,"+",y,"=",res,"c=",cbit,"lt=",lt)
    return res

def SPDZ_prepare(beta1, beta2, group_size):
    """
    Function to produce Beaver triple shares.
    """
    U, V = random.randint(3, 6), random.randint(3, 6)
    W = U * V
    return share(U, group_size), share(V, group_size), share(W, group_size), share(beta1, group_size), share(beta2, group_size)

def SPDZ_execute(preparation_data, group_size):
    """
    Function to execute the SPDZ protocol.
    """
    with concurrent.futures.ThreadPoolExecutor() as executor:
        barrier = Barrier(group_size)
        U_shares, V_shares, W_shares, x_shares, y_shares = preparation_data
        queues = [queue.Queue() for _ in range(group_size)]
        futures = [
            executor.submit(
                CP, j, x_shares[j], y_shares[j], U_shares[j], V_shares[j], W_shares[j], 
                queues[j], [queues[k] for k in range(group_size) if k != j], barrier
            ) for j in range(group_size)
        ]

        group_results = [f.result() for f in futures]
        print("Final results share of each party:", group_results)
        sum_total = sum(group_results)
        print("Final Result:", sum_total)

# Main execution
if __name__ == "__main__":
    input1, input2 = 20, 10  # The multiplication input numbers
    client_num = 5  # Number of clients

    preparation_data = SPDZ_prepare(input1, input2, client_num)
    SPDZ_execute(preparation_data, client_num)


A= [0, 0, 0, 1, 1]
B= [0, 0, 1, 1, 0]
S= [0, 1, 0, 0, 1] cbit= 0
3 + 6 = 9 c= 1 lt= 1
A= [0, 0, 0, 1, 0]
B= [0, 0, 1, 0, 1]
S= [0, 0, 1, 1, 1] cbit= 0
2 + 5 = 7 c= 0 lt= 1
A= [0, 0, 1, 1, 0]
B= [0, 0, 0, 1, 0]
S= [0, 1, 0, 0, 0] cbit= 0
6 + 2 = 8 c= 1 lt= 0
A= [0, 0, 0, 1, 1]
B= [0, 0, 1, 0, 1]
S= [0, 1, 0, 0, 0] cbit= 0
3 + 5 = 8 c= 1 lt= 1
A= [0, 0, 0, 1, 1, 0]
B= [1, 1, 1, 0, 0, 0]
S= [1, 1, 1, 1, 1, 0] cbit= 0
6 + -8 = -2 c= 0 lt= 1
Final results share of each party: [9, 7, 8, 8, -2]
Final Result: 30
